In [1]:
import pyspark
sc = pyspark.SparkContext(appName="AirbnbSparkonHDF")

24/05/21 21:45:56 WARN Utils: Your hostname, boui resolves to a loopback address: 127.0.1.1; using 192.168.148.131 instead (on interface ens33)
24/05/21 21:45:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/21 21:45:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/21 21:45:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/05/21 21:45:57 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/05/21 21:45:57 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

/home/boui/Desktop/github/2024-01-BIGDATA-PROJECT2/venv/lib/python3.10/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
hdfs_path = "hdfs://localhost:9000/user/hadoop/"
file_path = hdfs_path + "airbnb.csv"

df_air = sqlContext.read.csv(path=file_path, header=True, inferSchema=True)

In [4]:
df_air = df_air.fillna({'overall_satisfaction':3.0})

In [5]:
df_air = df_air.withColumn('total_score', df_air.overall_satisfaction + df_air.reviews * 0.378)

In [6]:
df_air1 = df_air.select('room_id', 'host_id', 'total_score')
from pyspark.sql.functions import struct, col
df_air1 = df_air1.withColumn("room_id_str", df_air1.room_id.cast("string"))
df_air1 = df_air1.withColumn("host_id_str", df_air1.host_id.cast("string"))

In [7]:
from pyspark.sql.functions import concat_ws
df_air1 = df_air1.withColumn("room_id, host_id", concat_ws(',', df_air1.room_id_str, df_air1.host_id_str)) \
            .select('room_id, host_id', 'total_score')

In [8]:
df_air1 = df_air1.orderBy('total_score', ascending = True)
df_air1.show()

+-----------------+-----------+
| room_id, host_id|total_score|
+-----------------+-----------+
|11757251,26873897|      2.134|
|12186553,15589117|        3.0|
| 2513870,12867663|        3.0|
| 6551454,29224952|        3.0|
|10153739,30283594|        3.0|
|12808014,28197086|        3.0|
|12897590,70637430|        3.0|
| 5824015,30228015|        3.0|
|12563549,34226261|        3.0|
|13007354,20696611|        3.0|
|12890140,54064917|        3.0|
|10118379,30283594|        3.0|
|11432300,60063253|        3.0|
|13062783,44620221|        3.0|
|10053369,30283594|        3.0|
|11256475,54064917|        3.0|
| 7052847,10336060|        3.0|
| 9992532,30283594|        3.0|
|  2443944,4442258|        3.0|
| 12254824,2086352|        3.0|
+-----------------+-----------+
only showing top 20 rows



In [9]:
df_air1.write.csv(hdfs_path + 'sorted_total_score_ascend.csv', header=True)

In [10]:
df_air1 = df_air1.orderBy('total_score', ascending = False)
df_air1.show()

+----------------+-----------+
|room_id, host_id|total_score|
+----------------+-----------+
|    66288,324630|     149.53|
|  414419,2027295|    111.474|
| 1497879,2776892|    110.718|
|    31796,119019|    103.536|
|  815639,3637081|     98.622|
|  916123,3637081|     98.622|
| 1136972,1480518|     97.232|
|    47521,119019|      89.55|
|  1695275,324630|     88.294|
|  2776143,324630|     84.892|
|   197972,965697|     84.002|
|  766700,3637081|     83.502|
| 1615033,3637081|     82.746|
|     22354,85770|     81.234|
| 1141522,1407005|     81.234|
| 1472520,6608084|     80.222|
| 1472481,6608084|     79.844|
|  708802,3648427|     79.844|
| 1147871,1407005|     78.966|
| 1544702,6608084|      78.71|
+----------------+-----------+
only showing top 20 rows



In [11]:
df_air1.write.csv(hdfs_path + 'sorted_total_score_descend.csv', header=True)